In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!tar -xvf "/content/drive/MyDrive/cifar-10-python.tar.gz"

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

train_data = None
train_labels = []
data_dir='cifar-10-batches-py'

for i in range(1, 6):
 with open(data_dir + "/data_batch_{}".format(i), 'rb') as f:
    data_dic = pickle.load(f, encoding='latin-1')
 if i == 1:
  train_data = data_dic['data']
 else:
  train_data = np.vstack((train_data, data_dic['data']))
 train_labels += data_dic['labels']
 with open(data_dir + "/data_batch_{}".format(i), 'rb') as f:
  test_data_dic = pickle.load(f, encoding='latin-1')
test_data = test_data_dic['data']
test_labels = test_data_dic['labels']

train_data = train_data.reshape((len(train_data), 3, 32, 32))
train_data = np.rollaxis(train_data, 1, 4)
train_labels = np.array(train_labels)

test_data = test_data.reshape((len(test_data), 3, 32, 32))
test_data = np.rollaxis(test_data, 1, 4)
test_labels = np.array(test_labels)

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)

# In order to check where the data shows an image correctly
with open(data_dir + "/batches.meta", 'rb') as f:
    label_encoding  = pickle.load(f, encoding='latin-1')
label_encoding = label_encoding['label_names']

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [ ]:
import tensorflow.keras as keras
train_data = train_data.astype("float32")/255.0
test_data = test_data.astype("float32")/255.0
# contains 10 classes
train_labels = keras.utils.to_categorical(train_labels,10)
test_labels = keras.utils.to_categorical(test_labels,10)

In [ ]:
# import the necessary pacakges
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
import keras.layers.normalization.batch_normalization as BatchNormalization
from keras.layers.convolutional import MaxPooling2D as MaxPool2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from tensorflow import  keras


from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(10, activation='softmax'))
adam = Adam(lr=0.001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy'])
from keras.preprocessing.image import ImageDataGenerator

trainGen = ImageDataGenerator()
result = model.fit_generator(trainGen.flow(train_data,train_labels,batch_size = 512),validation_data = (test_data,test_labels),epochs=30,steps_per_epoch= 40)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


40/40 [==============================] - 419s 10s/step - loss: 2.0822 - accuracy: 0.2216 - val_loss: 1.8530 - val_accuracy: 0.3242
Epoch 2/30
40/40 [==============================] - 422s 11s/step - loss: 1.7014 - accuracy: 0.3742 - val_loss: 1.6100 - val_accuracy: 0.4114
Epoch 3/30
40/40 [==============================] - 420s 10s/step - loss: 1.5429 - accuracy: 0.4302 - val_loss: 1.4968 - val_accuracy: 0.4434
Epoch 4/30
40/40 [==============================] - 424s 11s/step - loss: 1.3733 - accuracy: 0.4976 - val_loss: 1.2650 - val_accuracy: 0.5462
Epoch 5/30
40/40 [==============================] - 427s 11s/step - loss: 1.2770 - accuracy: 0.5376 - val_loss: 1.2264 - val_accuracy: 0.5622
Epoch 6/30
40/40 [==============================] - 425s 11s/step - loss: 1.1503 - accuracy: 0.5903 - val_loss: 1.0690 - val_accuracy: 0.6179
Epoch 7/30
40/40 [==============================] - 424s 11s/step - loss: 1.0618 - accuracy: 0.6188 - val_loss: 1.0180 - val_accuracy: 0.6303
Epoch 8/30
40/40 

In [ ]:
import matplotlib.pyplot as plt
acc_values = result.history['accuracy']
val_acc_values = result.history['val_accuracy']
plt.plot(epochs,acc_values,'bo',label='Training accuracy')
plt.plot(epochs,val_acc_values,'b',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
##inception Block
from keras import layers
x = Input(shape = (32,32,3))
branch_a = layers.Conv2D(128,1,activation='relu',strides = 2)(x)
branch_b = layers.Conv2D(128,1,activation='relu')(x)
branch_b = layers.Conv2D(128,3,activation='relu',strides=2)(branch_b)
branch_c = layers.AveragePooling2D(3,strides=2)(x)
branch_c = layers.Convo2D(128,3,activation='relu')(branch_c)
branch_d = layers.Convo2D(128,1,activation='relu')(x)
branch_d = layers.Conv2D(128,3,activation='relu')(branch_d)
branch_d = layers.Conv2D(128,3,activation='relu',strides=2)(branch_d)
model = layers.concatenate(branch_a,branch_b,branch_c,branch_d,axis=-1)
result = model.fit_generator(trainGen.flow(train_data,train_labels,batch_size = 512),validation_data = (test_data,test_labels),epochs=30,steps_per_epoch= 40)

In [ ]:
import matplotlib.pyplot as plt
acc_values = result.history['accuracy']
val_acc_values = result.history['val_accuracy']
plt.plot(epochs,acc_values,'bo',label='Training accuracy')
plt.plot(epochs,val_acc_values,'b',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#Residual Block
x =  Input(shape = (32,32,3))
y = layers.convo2D(128,3,activation='relu',padding='same')(x)
y = layers.convo2D(128,3,activation='relu',padding='same')(y)
y = layers.convo2D(128,3,activation='relu',padding='same')(y)
residual = layers.Convo2D(128,1,strides=2)(y)
model = layers.add([y,residual])
result = model.fit_generator(trainGen.flow(train_data,train_labels,batch_size = 512),validation_data = (test_data,test_labels),epochs=30,steps_per_epoch= 40)

In [ ]:
import matplotlib.pyplot as plt
acc_values = result.history['accuracy']
val_acc_values = result.history['val_accuracy']
plt.plot(epochs,acc_values,'bo',label='Training accuracy')
plt.plot(epochs,val_acc_values,'b',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()